In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pathlib import Path
from scipy.optimize import curve_fit
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection, PolyCollection
from os.path import dirname, join as pjoin
from scipy.optimize import minimize
import scipy.io as sio
from scipy.io import readsav
import glob
plt.rcParams.update({'font.size': 16})

In [ ]:
#path to where data are being stored (from h3ppy fits)
path = r"Y:\obs_23\Keck_29Dec\Kate's Final Temperatures & Densities (identical)"
date_fortitles = "Dec 30, 2023" #just used in the titles of plots

In [ ]:
#load scamspec - contains locational information for the data
scamspecfile = sio.readsav(r"Y:\obs_23\Keck_29Dec\spec\scamspec_22Mar2024.sav", verbose=False, python_dict = True)
scamspec = scamspecfile['scamspec'][0]

In [ ]:
scamspec["SPEC_FILE"][0] #check which data file scamspec starts on - adjust accordingly with your data

In [ ]:
startindex = 0 #change 0 to index needed if scamspec starts on a different file

#longitudes
longs_array = scamspec['AVG_LONGS']
longs_df = pd.DataFrame(longs_array[:,startindex:]) 

#latitudes
avg_lats_pc = scamspec["AVG_LATS_PC"]
lats_df = pd.DataFrame(avg_lats_pc[:,startindex:])

#emission angle
emi = pd.DataFrame(scamspec["AVG_EMI"][:,startindex:])

In [ ]:
#load in the lie of sight correction for future implementation on the densities
LOS_corr = np.load("chapman_correction.npy")
angle = LOS_corr[0]
corr = LOS_corr[1]


#array of possible angles to interpolate to - in this case we have a range from 0 to 100 in steps of 0.01
angle_highres = np.arange(0,100,0.01)

#interpolating data to the size array that's easiest to work with
coor_interp = np.interp(angle_highres, angle, corr)

In [ ]:
newpath = path + "/latitudes" #directory where new files with long-lat information will be stored
if not os.path.exists(newpath):
    os.makedirs(newpath)


index = 0
your_path = path
files = os.listdir(your_path)
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f_name, f_ext = os.path.splitext(os.path.join(your_path, file))
        jup = pd.read_csv(os.path.join(your_path, file)) #open data file
        density = jup["Densities"] #first opening densities to apply LOS correction
        density_err = jup["Density Error"]
        
        corr_dens_list = []
        corr_dens_err_list = []
        for i in range(len(density)):
            emi_angle = emi[index][i]

            emi_index = np.abs(angle_highres - np.abs(emi_angle)).argmin() #find index of closest angle to emission angle
            density_corr = density[len(density)-1 - i] / coor_interp[emi_index] #applying LOS correction to density
            density_err_corr = density_err[len(density_err)-1 - i] / coor_interp[emi_index] #" " and density error
            corr_dens_list.append(density_corr) 
            corr_dens_err_list.append(density_err_corr)
        
        corr_dens_rev = corr_dens_list[::-1] #reverse direction of list
        corr_dens_err_rev = corr_dens_err_list[::-1] #" "
           
        print("Index: " + str(index) + " and path:" + str(os.path.join(your_path, file)))
        
        juplats = lats_df[index] #define array of latitudes
        print(Path(f_name).stem, str(np.median(juplats)))
        juplatsrev = juplats[::-1] #reverse direction cause they're backwards
        #print(np.median(juplats))
        
        juplongs = longs_df[index] #re: above
        juplongsrev = juplongs[::-1]
        
               
        jup.insert(4, "Latitude_pc", np.asarray(juplatsrev), True) #insert lat array into larger df
            
        jup.insert(4, "Longitude_avg", np.asarray(juplongsrev), True) #insert long array into larger df
        
        jup.insert(4, "Corrected_density_error", np.asarray(corr_dens_err_rev), True) #insert corrected density array into larger df
        
        jup.insert(4, "Corrected_density", np.asarray(corr_dens_rev), True) #insert corrected density error array into larger df
             
    
        #print(jup_final)
        #save final version of the dataframe as a csv file
        jup.to_csv(newpath + Path(f_name).stem + "_lats.csv", encoding='utf-8', index=False)
        index = index + 1

In [ ]:
#function to go through files one at a time to make sure everything looks reasonable

your_path = newpath #path where files were saved above
files1 = os.listdir(your_path)

#this bit is creating a list of all the file names so the function can call both the final temperatures/densities but also from the directory with fits
name_arr = []
count1 = 0
for file1 in files1:
    if os.path.isfile(os.path.join(your_path, file1)):
        f_name, f_ext = os.path.splitext(os.path.join(your_path, file1))
        stringname = Path(f_name).stem
        editable = stringname.split('_')
        partialname = str(editable[0] + "_" + editable[1])
        name_arr.append(partialname)
        count1 = count1 + 1
print(count1)


def plotting_func(num):

    #these might need to be changed to fit your naming conventions/paths
    jup = pd.read_csv(newpath + "/"+ name_arr[num] + "_temps_lats.csv", header = 0) #path to above files
    jupfit = np.load(path + "/fits/" +  name_arr[num] +"_fits.npy") #path to data fits from h3ppy
    print(name_arr[num])



    plt.rcParams['axes.facecolor'] = 'white'  

    #Create a x scale for plotting 
    xx      = list(range(len(jupfit[0][0])))
    xaxis = []
    for i in range(len(xx)):
        xaxis.append(xx[i])

    truecenters_multi = [3.41488, 3.42071, 3.45475, 3.95299] #for labeling the emission lines in the plot
    cpos = np.arange(len(truecenters_multi)) * 28 + 15

    
#___________________________________________________#
    #these values can be changed to view specific fits within the data file
    slicein1 = 151
    slicein2 = 30
#___________________________________________________#

    
    plt.rcParams['axes.facecolor'] = 'white'


    fig, ax = plt.subplots(figsize=[7,3])
    plt.title("Cuthrough showing Fit and Residual")
    plt.plot(xaxis, (np.asarray(jupfit[0]))[slicein1], label = "Data", color = "red", linewidth = 2)
    plt.plot(xaxis, (np.asarray(jupfit[1]))[slicein1], label = "h3ppy Fit, temp = "+ str(round(jup["Temperatures"][slicein1])) + "K", color = "tab:orange")
    #plt.plot(xaxis, (np.asarray(jupfit[0]) - np.asarray(jupfit[1]))[slicein1], label = "Residual", color = "tab:green")
    ax.set(xticks = cpos)
    ax.set_xticklabels(truecenters_multi)
    plt.legend()
    plt.show()
    
    fig, ax = plt.subplots(figsize=[7,3])
    plt.title("Cuthrough showing Fit and Residual")
    plt.plot(xaxis, (np.asarray(jupfit[0]))[slicein2], label = "Data", color = "blue", linewidth = 2)
    plt.plot(xaxis, (np.asarray(jupfit[1]))[slicein2], label = "h3ppy Fit, temp = "+ str(round(jup["Temperatures"][slicein2])) + "K", color = "tab:orange")
    #plt.plot(xaxis, (np.asarray(jupfit[0]) - np.asarray(jupfit[1]))[slicein2], label = "Residual", color = "tab:green")
    ax.set(xticks = cpos)
    ax.set_xticklabels(truecenters_multi)
    plt.legend()
    plt.show()



    maxerror = []
    for i in range(len(jup["Temperatures"])):
        if jup["Temperatures"][i] == -666 or jup["Temperature Error"][i] == -666 or np.abs(jup["Latitude_pc"][i]) > 90:
            maxerror.append(np.nan)
            if jup["Temperatures"][i] == -666 and np.abs(jup["Latitude_pc"][i]) < 90:
                print("666 Index: ", i)
        else:
            maxerrorvalue = jup["Temperatures"][i] + jup["Temperature Error"][i]
            maxerror.append(maxerrorvalue)
    maxerror_arr = np.asarray(maxerror)

    minerror = []
    for i in range(len(jup["Temperatures"])):
        if jup["Temperatures"][i] == -666 or jup["Temperature Error"][i] == -666 or np.abs(jup["Latitude_pc"][i]) > 90:
            minerror.append(np.nan)
        else:
            minerrorvalue = jup["Temperatures"][i] - jup["Temperature Error"][i]
            minerror.append(minerrorvalue)
    minerror_arr = np.asarray(minerror)

    plt.rcParams['axes.facecolor'] = 'white'  

    plt.subplots(figsize=[16, 6])



    plt.fill_between(jup["Latitude_pc"], minerror_arr, maxerror_arr, color = "wheat", alpha = 1)
    plt.scatter(jup["Latitude_pc"], jup["Temperatures"], color = "brown")
    plt.axvline(x = jup["Latitude_pc"][slicein1], color = "red")
    plt.axvline(x = jup["Latitude_pc"][slicein2], color = "blue")




    #plt.axvline(x = -23, color = "r", label = "Approximate GRS Boundaries")
    #plt.axvline(x = -11, color = "r")

    plt.title("CML " + str(round(np.median(jup["Longitude_avg"]),2)) + " ("+ date_fortitles + "); file: " + name_arr[num])
    plt.xlabel("Latitude$_{PC}$ (°)")
    #plt.tight_layout()
    plt.ylabel("Temperature $H_3^+$ (K)")
    #plt.grid(visible = True)

    if "S" in name_arr[num] or 's' in name_arr[num]:
        plt.xlim(-90,10)
        #plt.legend(loc = "lower left")
        #print("South")
    elif "N" in name_arr[num] or "n" in name_arr[num]:
        plt.xlim(-10,90)
        #plt.legend(loc = "lower right")
        #print("North")

    plt.ylim(500, 2000)

    #plt.savefig('C:/Users/rober/Pictures/Research Figures/epsc 2023 figures/GRS temp fit.png')
    #plt.savefig('C:/Users/kater00/Documents/Research Figures/GRS temp fit.png')

    plt.show()

In [ ]:
#start with whatever position you want, I'd reccomend count = 0
#then comment out count and rerun the cell to progress through all of the data files
#if there's a file you want to look at multiple fits within (if you have an extreme outlier):
        #1: you can change the indices for the fits in the above function
        #2: you can set the count to the number showing (because count is printed each time at the top of the graphs)
        #3: if you leave the count uncommented with the value of the file you want to look at, you can rerun the cell as many times as you need
count = 0

print("Count:", count)
plotting_func(count)
count = count + 1

In [ ]:
#once you're happy with all of the fits you can move onto plotting them

In [ ]:
#load all of the data into big lists

lats = []
longs = []
temps = []
temperr = []
dens = []
denerr = []

lats_big = []
longs_big = []

index = 0
your_path = newpath #path from above where files were saved
files = os.listdir(your_path)
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        jup = pd.read_csv(os.path.join(your_path, file)) #open data file
        jup_lats = jup["Latitude_pc"]
        jup_longs = jup["Longitude_avg"]
        jup_temps = jup["Temperatures"]
        jup_temperr = jup["Temperature Error"]
        jup_dens = jup["Densities"]
        jup_denerr = jup["Density Error"]
        
        
        
        for i in range(len(jup_temps)):
            lats_big.append(jup_lats[i]) #these used in the next cell for intensities
            longs_big.append(jup_longs[i])

            #skips any "obviously wrong data" including the dummy variables put in during h3ppy fits and off-planet values from the lat-long mapping
            if jup_temps[i] == -666 or jup_temperr[i] == 666 or \
               jup_temps[i] == np.nan or jup_temperr[i] == np.nan or \
               jup_dens[i]  == -666 or jup_denerr[i] == 666 or \
               jup_dens[i]  == np.nan or jup_denerr[i] == np.nan or \
               np.abs(jup_lats[i]) > 90 or np.abs(jup_longs[i]) > 360:
                continue
            else:
                lats.append(jup_lats[i])
                longs.append(jup_longs[i])
                temps.append(jup_temps[i])
                temperr.append(jup_temperr[i])
                dens.append(jup_dens[i])
                denerr.append(jup_denerr[i])

In [ ]:
#this is specifically for mapping intensities 
#need all of the lats/longs from before because the fits (where the intensity value comes from) haven't been cropped
peak_value = []
your_path = "Y:/obs_23/Keck_29Dec/Kate's Final Temperatures & Densities (identical)/fits" #or wherever you're keeping your fits from the h3ppy routines
files = os.listdir(your_path)
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        jup = np.load(os.path.join(your_path, file))
        
        data = jup[0]
        for i in range(len(data)):
            value = data[i][-15]
            peak_value.append(value)
            
peak_value_short = []
longs_big_short = []
lats_big_short = []

for i in range(len(peak_value)):
    if np.abs(lats_big[i]) > 90 or np.abs(longs_big[i]) > 360: #now the bad values from above can be dropped
        continue
    else:
        #print(lats_big[i])
        peak_value_short.append(peak_value[i])
        lats_big_short.append(lats_big[i])
        longs_big_short.append(longs_big[i])

In [ ]:
plt.subplots(figsize=[7, 10])

plt.rcParams.update({'lines.markersize': 5})
plt.rcParams['axes.facecolor'] = 'darkgrey'


#update colorbar bounds to improve contrast if necessary
vmin = 1e-3
vmax = 3e-2

markersize = np.full(
  shape=len(longs_big_short),
  fill_value=40,
  dtype=int
)

colors_cube = plt.get_cmap('gist_stern')(np.linspace(0, 1, 7))
cmap_segmented = LinearSegmentedColormap.from_list('cubehelix_segmented', colors_cube, N = 11)

scatter = plt.scatter(longs_big_short, lats_big_short, s = markersize, c = peak_value_short, cmap = cmap_segmented, 
                       vmin = vmin, vmax = vmax, linewidth = 0, marker = "*")
cbar = plt.colorbar(scatter, shrink = 0.9, extend = "both")
cbar.set_label(r"Intensity ($W/m^2/\mu m/sr$)")


plt.xlim(60,217) #update bounds for longs
plt.gca().invert_xaxis()
plt.minorticks_on()



plt.title(date_fortitles, color = "purple")
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
plt.show()

In [ ]:
#change the variable names in case you accidently rerun something above 
#(also cause I apparently changed my names somehwere along the way and this is the solution I've been living with)
lats_arr = lats
longs_arr = longs
temps_arr = temps
temperr_arr = temperr
dens_arr = dens
denerr_arr = denerr

In [ ]:
#code for creating data bins for plotting
#all this does is create literal boxes in lat and long - the loops look for lat-long points of data which fall inside a region
#if there's a point, it gets added to a list; once all the data have been searched, the loop takes a bootstrap median of the temp and density
#then the root mean square of the uncertainties for everything in the bin as well as the standard deviation
#those 3 values then get appended to their respctive lists, and the loop makes the next bin and continues the search

step_lat = 3 #can update bins to whatever size you want - this was just what we ended up using
step_long = 6
bounds_list_longs = list(range(50, 360 - step_long, step_long))
bounds_list_lats = list(range(-90, 90 - step_lat, step_lat))

long_up = []
long_down = []
lat_small = []
lat_large = []

temp_lats = []
terr_lats = []
tstd_lats = []
dens_lats = []
derr_lats = []
dstd_lats = []

# Function to generate bootstrapped samples and compute medians
def bootstrap_median(values, errors, n_bootstraps = 1000):
    medians = []
    for _ in range(n_bootstraps):
        # Generate a sample considering the errors
        sample = np.random.normal(values, errors)
        medians.append(np.median(sample))
    return np.array(medians)

for bound_long in bounds_list_longs:
    for bound_lat in bounds_list_lats:
        temp_list = []
        dens_list = []
        temp_err_list = []
        dens_err_list = []

        for i in range(len(lats)):
            if lats[i] > bound_lat and lats[i] < bound_lat + step_lat \
                and longs[i] > bound_long and longs[i] < bound_long + step_long:
                
                temp_list.append(temps_arr[i])
                temp_err_list.append(temperr_arr[i])
                dens_list.append(dens_arr[i])
                dens_err_list.append(denerr_arr[i])
                
        if len(temp_list)>0:
            temp_median = np.median(bootstrap_median(temp_list, temp_err_list)) #median value from bootstrap
            dens_median = np.median(bootstrap_median(dens_list, dens_err_list)) # " "        
            temp_err = np.sqrt(sum(np.asarray(temp_err_list)**2)/len(temp_err_list)) #RMS error
            dens_err = np.sqrt(sum(np.asarray(dens_err_list)**2)/len(dens_err_list)) #" "
            temp_std = np.std(temp_list) #standard deviation
            dens_std = np.std(dens_list) #"  "
                     
            
            temp_lats.append(temp_median)
            dens_lats.append(dens_median)
            terr_lats.append(temp_err)
            derr_lats.append(dens_err)
            tstd_lats.append(temp_std)
            dstd_lats.append(dens_std)
        elif len(temp_list) == 0: #if no values fall within a given bin, that bin is filled with nan
            temp_lats.append(np.nan)
            dens_lats.append(np.nan)
            tstd_lats.append(np.nan)
            dstd_lats.append(np.nan)
            

        #saving the "four corners" for every bin created
        long_down.append(bound_long)
        long_up.append(bound_long + step_long)
        lat_small.append(bound_lat)
        lat_large.append(bound_lat + step_lat)

In [ ]:
#load in dark ribbon and auroral coordinates
darkribbon = pd.read_csv(r"Y:\obs_22\H3+ Dark Ribbon Coordinates Stallard et al.csv", header = None)
outer = pd.read_csv(r"Y:/tools/auroral ovals/outer ovals.csv", dtype=np.float64)


#if you want to plot the location of the GRS
# Define the equation of an ellipse
def ellipse_eq(params, x, y):
    x0, y0, a, b, angle = params
    theta = np.deg2rad(angle)
    x_rot = (x - x0) * np.cos(theta) - (y - y0) * np.sin(theta)
    y_rot = (x - x0) * np.sin(theta) + (y - y0) * np.cos(theta)
    return np.sum(((x_rot / a) ** 2 + (y_rot / b) ** 2 - 1) ** 2)


GRSwidth = 14.48 #width in longitude, degrees

#create line to map GRS - GRS drifts in longitude, create new ones
# Initial guess for the parameters
#this guess is [center longitude, center latitude, and then some stuff that I wouldn't change]
initial_params = [101.8425, -17, 10, 10, 0]  # all you really need to change is the first number (center longitude, degrees W)


data_points = np.array([
    [initial_params[0], -23],
    [initial_params[0], -11],
    [initial_params[0] - GRSwidth/2, initial_params[1]],
    [initial_params[0] + GRSwidth/2, initial_params[1]]
])

# Fit the ellipse to the data points
result = minimize(ellipse_eq, initial_params, args=(data_points[:, 0], data_points[:, 1]))

# Extract the fitted parameters
x0, y0, a, b, angle = result.x

# Create points for the ellipse
theta_GRS = np.linspace(0, 2 * np.pi, 100)
x_fit_GRS = x0 + a * np.cos(theta_GRS)
y_fit_GRS = y0 + b * np.sin(theta_GRS)

In [ ]:
#plot map as individual sactter points:
#(points are center of pixel along slit and not accounting for width of slit or seeing)

In [ ]:
plt.subplots(figsize=[7, 10]) #change figure size, [width, height]

plt.rcParams.update({'lines.markersize': 5})
plt.rcParams['axes.facecolor'] = 'gray'


#bounds for temperatures - change to improve contrast
vmin = 650
vmax = 1350

#you can always make your own colormap too if you want - I use this site:
#https://gka.github.io/palettes/#/9|s|00429d,96ffea,ffffe0|ffffe0,ff005e,93003a|1|1

temp_cmap = ListedColormap(['#1d2949', '#2c3361', '#3e3d76', '#544786', '#6a5291', '#815e97', '#966b99', \
                            '#aa7a97', '#bc8993', '#cd9a8c', '#dbac84', '#e8be7b', '#f4d170', '#ffe463'])

#data
markersize = np.full(
  shape=len(longs_arr),
  fill_value=40, #can make markers bigger or smaller with this
  dtype=int
)

scatter = plt.scatter(longs_arr, lats_arr, s = markersize, c = temps_arr, cmap = temp_cmap, 
                       vmin = vmin, vmax = vmax, linewidth = 0, marker = "*")


#colorbar info
cbar = plt.colorbar(scatter, shrink = 0.9, extend = "max")
cbar.set_label("Column-Integrated Temperature (K)")

#locational indicators
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black") #the southern oval plots weird wihtout splitting - just go with it
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")

plt.plot(x_fit_GRS, y_fit_GRS, label='GRS: ' + date_fortitles, color='white', alpha = 0.7)


plt.xlim(60,217) #change for longitude bounds


plt.gca().invert_xaxis()
plt.minorticks_on()
plt.title(date_fortitles, color = "purple")
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
plt.show()

In [ ]:
#plot using bins from above:

In [ ]:
fig,ax = plt.subplots(figsize=[7, 10])


polygons = []

cmap = temp_cmap
norm = matplotlib.colors.Normalize(650, 1350) #change colormap bounds here

for i in range(len(long_down)):
    verts = [
        (long_down[i], lat_small[i]),
        (long_down[i], lat_large[i]),
        (long_up[i], lat_large[i]),
        (long_up[i], lat_small[i])
            ]
    polygons.append(verts)

collection = PolyCollection(polygons, cmap=cmap, norm=norm, edgecolors='None') #calls colormap from above

collection.set_array(temp_lats)

ax.add_collection(collection)
ax.autoscale_view()
plt.colorbar(collection, ax=ax, label='Column-Integrated Temperature (K)', extend = "max")

#locational indicators
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black") #the southern oval plots weird wihtout splitting - just go with it
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")

plt.plot(x_fit_GRS, y_fit_GRS, label='GRS: ' + date_fortitles, color='white', alpha = 0.7)


plt.xlim(60,217)
plt.gca().invert_xaxis()
plt.minorticks_on()
plt.title(date_fortitles, color = "purple")
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
plt.show()

In [ ]:
#now onto densities

In [ ]:
plt.subplots(figsize=[6, 10])

plt.rcParams.update({'lines.markersize': 5})
plt.rcParams['axes.facecolor'] = 'gray'


vmin = 1e15
vmax = 5e15

dens_cmap = ListedColormap(['#692038', '#8c354d', '#a1575a', '#ab7d62', '#b3a26b', '#bec57b', '#d5e497', '#fdffc1'])


markersize = np.full(
  shape=len(longs_arr),
  fill_value=40,
  dtype=int
)

scatter = plt.scatter(longs_arr, lats_arr, s = markersize, c = dens_arr, cmap = dens_cmap, 
                       vmin = vmin, vmax = vmax, linewidth = 0, marker = "*")



cbar = plt.colorbar(scatter, extend = "both", shrink = 0.9)
cbar.set_label("Column-Integrated Density ($m^{-2}$)")

#including locational indicators (GRS, GCS, auroral ovals, h3+ dark ribbon)
plt.plot(x_fit_GRS, y_fit_GRS, label='GRS Dec 29 2023', color='white', alpha = 0.7)
plt.scatter(darkribbon[1], darkribbon[0], label = "$H_3^+$ Dark Ribbon", color = "white", marker = "x")
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black")
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")


plt.xlim(60,217)

plt.gca().invert_xaxis()
plt.title('$H_3^+$ Densities: ' +  date_fortitles)
plt.xlabel("CML (°W)")
plt.ylabel('Latitude$_{PC}$')
plt.legend(bbox_to_anchor=(0.5, -0.1), ncol = 1)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=[7, 10])


polygons = []

cmap = dens_cmap
norm = matplotlib.colors.Normalize(1e15, 5e15) #change colormap bounds here

for i in range(len(long_down)):
    verts = [
        (long_down[i], lat_small[i]),
        (long_down[i], lat_large[i]),
        (long_up[i], lat_large[i]),
        (long_up[i], lat_small[i])
            ]
    polygons.append(verts)

collection = PolyCollection(polygons, cmap=cmap, norm=norm, edgecolors='None') #calls colormap from above

collection.set_array(dens_lats)

ax.add_collection(collection)
ax.autoscale_view()
plt.colorbar(collection, ax=ax, label='Column-Integrated Density ($m^{-2}$)', extend = "both")

#locational indicators
plt.plot(x_fit_GRS, y_fit_GRS, label='GRS Dec 29 2023', color='white', alpha = 0.7)
plt.scatter(darkribbon[1], darkribbon[0], label = "$H_3^+$ Dark Ribbon", color = "white", marker = "x")
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black")
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")


plt.xlim(60,217)
plt.gca().invert_xaxis()
plt.minorticks_on()
plt.title(date_fortitles, color = "purple")
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
plt.show()

In [ ]:
#temp error, I'll leave making the binned plots as an exercise to the useer:

In [ ]:
plt.subplots(figsize=[6, 10])

plt.rcParams.update({'lines.markersize': 5})
plt.rcParams['axes.facecolor'] = 'gray'


vmin = 0
vmax = 8

markersize = np.full(
  shape=len(longs_arr),
  fill_value=40,
  dtype=int
)


temp_cmap = ListedColormap(['#283d80', '#64537b', '#8b6d7a', '#aa8981',\
                            '#c5a88e', '#ddc8a0', '#f4eab4', '#fffbbf'])

scatter = plt.scatter(longs_arr, lats_arr, s = markersize, c = (np.asarray(temperr_arr)/np.asarray(temps_arr))*100, \
                      cmap = "bone", vmin = vmin, vmax = vmax, linewidth = 0, marker = "*")
cbar = plt.colorbar(scatter, shrink = 0.9, extend = "max")
cbar.set_label("Root Mean Square Temperature Error (%)")

#including locational indicators (GRS, GCS, auroral ovals, h3+ dark ribbon)
#plt.plot(x_fit_GRS, y_fit_GRS, label='GRS Dec 29 2023', color='white', alpha = 0.7)
#plt.scatter(darkribbon[1], darkribbon[0], label = "$H_3^+$ Dark Ribbon", color = "white", marker = "x")
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black")
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")



plt.xlim(60,217)
plt.minorticks_on()

plt.gca().invert_xaxis()

#plt.title('$H_3^+$ Temperatures\nDec 15, 2022')
plt.title(date_fortitles)
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
#plt.legend(bbox_to_anchor=(0.5, -0.1), ncol = 1)
plt.show()

In [ ]:
#density error

In [ ]:
plt.subplots(figsize=[7, 10])

plt.rcParams.update({'lines.markersize': 5})
plt.rcParams['axes.facecolor'] = 'gray'


vmin = 0
vmax = 30

markersize = np.full(
  shape=len(longs_arr),
  fill_value=40,
  dtype=int
)


temp_cmap = ListedColormap(['#283d80', '#64537b', '#8b6d7a', '#aa8981',\
                            '#c5a88e', '#ddc8a0', '#f4eab4', '#fffbbf'])

scatter = plt.scatter(longs_arr, lats_arr, s = markersize, c = (np.asarray(denerr_arr)/np.asarray(dens_arr))*100, \
                      cmap = "pink", vmin = vmin, vmax = vmax, linewidth = 0, marker = "*")
cbar = plt.colorbar(scatter, shrink = 0.9, extend = "max")
cbar.set_label("Root Mean Square Density Error (%)")

#including locational indicators (GRS, GCS, auroral ovals, h3+ dark ribbon)
#plt.plot(x_fit_GRS, y_fit_GRS, label='GRS Dec 29 2023', color='white', alpha = 0.7)
#plt.scatter(darkribbon[1], darkribbon[0], label = "$H_3^+$ Dark Ribbon", color = "white", marker = "x")
plt.plot(outer["Lon_N"], outer["Lat_N"], color = "black")
plt.plot(outer["Lon_S"][0:2889], outer["Lat_S"][0:2889], color = "black")
plt.plot(outer["Lon_S"][2889:], outer["Lat_S"][2889:], color = "black")

plt.xlim(60,217)
plt.minorticks_on()

plt.gca().invert_xaxis()
plt.title(date_fortitles)
plt.xlabel("CML (°W)")
plt.ylabel('$Latitude_{PC}$')
#plt.legend(bbox_to_anchor=(0.5, -0.1), ncol = 1)
plt.show()